<a href="https://colab.research.google.com/github/visha1Sagar/Resume-Job_Description-Matching/blob/main/Resume_Job_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

dataset = load_dataset("cnamuangtoun/resume-job-description-fit")

train.csv:   0%|          | 0.00/53.4M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1759 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

label_map = {"No Fit": 0, "Potential Fit": 1, "Good Fit":2}  # Example mapping

def preprocess_function(examples):

    inputs= tokenizer(
        examples["resume_text"],
        examples["job_description_text"],
        truncation=True,
        padding="max_length",

        max_length=512
    )
    if "label" in examples:  # Ensure 'label' exists in the examples
        inputs["label"] = [label_map[label] for label in examples["label"]]

    return inputs

preprocessed_ds = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/6241 [00:00<?, ? examples/s]

Map:   0%|          | 0/1759 [00:00<?, ? examples/s]

In [15]:
train_dataset = preprocessed_ds["train"]
test_dataset = preprocessed_ds["test"]

In [19]:
print(train_dataset["label"][:10])  # Check first 10 labels


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/roberta-base",
    num_labels=3  # Number of classes in the dataset
)


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import os
os.environ["WANDB_MODE"] = "offline"


In [12]:
import os

# Set the environment variable
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Verify it's set
print(f"CUDA_LAUNCH_BLOCKING is set to: {os.environ.get('CUDA_LAUNCH_BLOCKING')}")


CUDA_LAUNCH_BLOCKING is set to: 1


In [21]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    evaluation_strategy="steps",
    learning_rate=2e-6,
    per_device_train_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.001,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
  )

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
10,1.029400,1.050115
20,1.064300,1.049117
30,1.022400,1.048464
40,1.035200,1.048437
50,1.023200,1.048794
60,1.032300,1.049149
70,1.073900,1.048417
80,1.027000,1.048114
90,1.053700,1.048094
100,1.024000,1.048082


TrainOutput(global_step=196, training_loss=1.039786942151128, metrics={'train_runtime': 1637.1485, 'train_samples_per_second': 3.812, 'train_steps_per_second': 0.12, 'total_flos': 1642090840040448.0, 'train_loss': 1.039786942151128, 'epoch': 1.0})

In [25]:
import torch

In [28]:
def predict(resume_text, job_description_text):
    """Performs inference on a single resume-job description pair or a batch."""

    inputs = tokenizer(
        resume_text,
        job_description_text,
        truncation=True,
        padding="max_length",
        max_length=512,  # Adjust max_length as needed
        return_tensors="pt"  # Convert to PyTorch tensors
    )
    device = next(model.parameters()).device  # Get model's device
    inputs = inputs.to(device)

    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)  # Forward pass through the model

    logits = outputs.logits  # Get logits (unnormalized scores)

    # Option 1: Predicted label index
    predicted_label_index = torch.argmax(logits, dim=-1).item()  # Get index of highest logit

    # Option 2: Predicted label (using label map)
    predicted_label = label_map[int(predicted_label_index)] if predicted_label_index in label_map else "Unknown"

    # Option 3: Softmax probabilities (optional)
    probabilities = torch.nn.functional.softmax(logits, dim=-1).squeeze().tolist()

    return predicted_label_index, probabilities  # Return label index (and optional probabilities)


In [41]:
trainer.evaluate()
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")


('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json',
 './fine_tuned_distilbert/tokenizer.json')